# V4
* 각각의 이미지가 (1000,1000)의 픽셀을 가져 너무 많은 연산에 의해 코드를 실행할 수 없으니 차원 축소
* Train Data에 적용하려면 시간이 오래 걸리므로 우선 Validation Data를 사용하여 각각의 이미지 픽셀 수 조정
* 우선 픽셀수만 변경해서 fit 시켜보고 메모리 문제 또 뜨면 각 이미지 gray 처리하자
* label을 문자열로 해놨었는데 int형으로 변환해주니 에러 발생하지 않음
* 이제 Training Data에 적용하러 V5로 가자

# GPU

In [1]:
import tensorflow.compat.v1 as tf1
# import tensorflow as tf2
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
config = tf1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.05
session = tf1.Session(config=config)

2022-11-05 15:15:21.955510: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-05 15:15:22.586515: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4053 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:b3:00.0, compute capability: 8.0


# Library

In [3]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Validation 파일안의 모든 이미지 픽셀수 변경 코드

In [5]:
# 원하는 주소 안에있는 png로 끝나는 파일들 모두 (28,28)로 형태 변환
def fixel(image_path):
    img_list = os.listdir(image_path)
    img_list_jpg = [img for img in img_list if img.endswith(".png")]

    lst = []
    for i in img_list_jpg:
        img = Image.open(image_path + i)
        img = img.resize((28,28))
        img_array = np.array(img)
        lst.append(img_array)
        
    return lst

In [6]:
Val_L = fixel("Data/Validation/L/")
Val_M = fixel("Data/Validation/M/")
Val_S = fixel("Data/Validation/S/")

In [8]:
X_val = Val_L + Val_M + Val_S

X_val_np = np.array(X_val)

X_val_np.shape

(1020, 28, 28, 3)

In [9]:
y_val = []

for i in ["L", "M", "S"]:
    for j in range(340):
        y_val.append(i)
        
y_val_np = np.array(y_val)
y_val_np.shape

(1020,)

# y_val_np 문자열을 정수로 변환

In [14]:
y_val_np = np.where(y_val_np == "L", 0, y_val_np)
y_val_np = np.where(y_val_np == "M", 1, y_val_np)
y_val_np = np.where(y_val_np == "S", 2, y_val_np)

y_val_np = y_val_np.astype("int")

# 데이터 나누기

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_val_np, y_val_np,
                                                    test_size=0.2, random_state=42)

# 학습
* 우선 실행되는지만 확인하기 위해 Validation Data를 사용하여 학습시켜본거임
* Training Data로 학습해보고 실행되면 이제 layer 조정할거임

In [18]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding="same"),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding="same"),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(32, activation='relu'),
    
    
  tf.keras.layers.Dense(10, activation='softmax')
])

model.summary()


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10)

loss, acc = model.evaluate(X_test, y_test, verbose=2)

print(loss, acc)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 26, 26, 16)        448       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 13, 13, 16)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 11, 11, 16)        2320      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 5, 5, 16)         0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 5, 5, 32)          4640      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 2, 2, 32)        

2022-11-05 15:18:44.461066: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8204
2022-11-05 15:18:45.196195: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-05 15:18:45.924753: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


26/26 [==============================] - 3s 12ms/step - loss: 2.1892 - accuracy: 0.3885
Epoch 2/10
26/26 [==============================] - 0s 10ms/step - loss: 0.9507 - accuracy: 0.5380
Epoch 3/10
26/26 [==============================] - 0s 10ms/step - loss: 0.8116 - accuracy: 0.6275
Epoch 4/10
26/26 [==============================] - 0s 10ms/step - loss: 0.6618 - accuracy: 0.7292
Epoch 5/10
26/26 [==============================] - 0s 10ms/step - loss: 0.5502 - accuracy: 0.7855
Epoch 6/10
26/26 [==============================] - 0s 10ms/step - loss: 0.5154 - accuracy: 0.7757
Epoch 7/10
26/26 [==============================] - 0s 10ms/step - loss: 0.4394 - accuracy: 0.8223
Epoch 8/10
26/26 [==============================] - 0s 10ms/step - loss: 0.3906 - accuracy: 0.8493
Epoch 9/10
26/26 [==============================] - 0s 10ms/step - loss: 0.3107 - accuracy: 0.8762
Epoch 10/10
26/26 [==============================] - 0s 10ms/step - loss: 0.3139 - accuracy: 0.8799
7/7 - 0s - loss: 0.4